In [237]:
import pandas as pd

In [238]:
# display floats with 2 decimal places
pd.options.display.float_format = '{:,.5f}'.format
 
# expand display limits
pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

In [239]:
df = pd.read_csv('all_stocks_2017-01-01_to_2018-01-01.csv',
                names=['Date', 'Open', 'High', 'Low', 'Close', 'Volume','Name']).reset_index().drop(0)
df.head()

,index,Date,Open,High,Low,Close,Volume,Name
1,1.00000,2017-01-03,178.83,180.0,177.22,178.05,2510055.0,MMM
2,2.00000,2017-01-04,178.03,178.9,177.61,178.32,1541985.0,MMM
3,3.00000,2017-01-05,178.26,179.14,176.89,177.71,1447848.0,MMM
4,4.00000,2017-01-06,177.29,178.6,175.8,178.23,1625049.0,MMM
5,5.00000,2017-01-09,178.37,178.38,177.2,177.27,1622625.0,MMM


In [240]:
for i in ['Open', 'High', 'Low', 'Close', 'Volume']:
    df.loc[:,i] = df.loc[:,i].astype('float')

In [241]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7781 entries, 1 to 7781
Data columns (total 8 columns):
index     7781 non-null float64
Date      7781 non-null object
Open      7757 non-null float64
High      7772 non-null float64
Low       7762 non-null float64
Close     7781 non-null float64
Volume    7781 non-null float64
Name      7781 non-null object
dtypes: float64(6), object(2)
memory usage: 547.1+ KB


In [242]:

print( df.Name.unique() )

['MMM' 'AXP' 'AAPL' 'BA' 'CAT' 'CVX' 'CSCO' 'KO' 'DIS' 'XOM' 'GE' 'GS'
 'HD' 'IBM' 'INTC' 'JNJ' 'JPM' 'MCD' 'MRK' 'MSFT' 'NKE' 'PFE' 'PG' 'TRV'
 'UTX' 'UNH' 'VZ' 'WMT' 'GOOGL' 'AMZN' 'AABA']


In [243]:
df_AAPL = df[df.Name.isin(['AAPL','GOOGL','MSFT','AMZN'])]
 

In [244]:
# pivot dataset

# calculate returns over the prior 7, 14, 21 and 28 days for the first day of each month
# pivot the dataset while keeping only one price column

pivoted_df = df_AAPL.pivot(index='Date', columns='Name', values='Close')

# display examples from pivoted dataset
pivoted_df.tail()

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2017-12-22,175.01000,"1,168.36000","1,068.86000",85.51000
2017-12-26,170.57000,"1,176.76000","1,065.85000",85.40000
2017-12-27,170.60000,"1,182.26000","1,060.20000",85.71000
2017-12-28,171.08000,"1,186.10000","1,055.95000",85.72000
2017-12-29,169.23000,"1,169.47000","1,053.40000",85.54000


In [245]:
# calculate returns over the prior 7, 14, 21 and 28 days using shift method

# shift the index of the dataframe by some number of periods
pivoted_df.tail(3)

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2017-12-27,170.60000,"1,182.26000","1,060.20000",85.71000
2017-12-28,171.08000,"1,186.10000","1,055.95000",85.72000
2017-12-29,169.23000,"1,169.47000","1,053.40000",85.54000


In [246]:
pivoted_df.tail(3).shift(1)

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2017-12-27,nan,nan,nan,nan
2017-12-28,170.60000,"1,182.26000","1,060.20000",85.71000
2017-12-29,171.08000,"1,186.10000","1,055.95000",85.72000


In [247]:
# to calcuate returns over 7 days prior we can use prices_today/prices_7_days_ago -1.0

#calculate returns over 7 days prior
delta_7 = pivoted_df / pivoted_df.shift(7) - 1.0

#display examples
delta_7.tail()

Name,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2017-12-22,0.01591,0.00363,0.01662,0.00187
2017-12-26,-0.00958,0.00213,0.00792,0.00838
2017-12-27,-0.01937,0.00265,-0.01101,-0.01313
2017-12-28,-0.03027,-0.00376,-0.02685,-0.00764
2017-12-29,-0.03042,-0.01508,-0.02443,-0.00338


In [248]:
# calculate returns over each window and store them in dictionary using loop

delta_dict = {}
for offset in [7, 14, 21, 28]:
    delta_dict['delta_{}'.format(offset)] = pivoted_df / pivoted_df.shift(offset) - 1.0

In [249]:
# melt data to create analytical base table (ABT)
# melt delta_7 returns

melted_7 = delta_7.reset_index().melt(id_vars='Date', value_name= 'delta_7')

#melted dataframe examples
melted_7.tail()

,Date,Name,delta_7
999,2017-12-22,MSFT,0.00187
1000,2017-12-26,MSFT,0.00838
1001,2017-12-27,MSFT,-0.01313
1002,2017-12-28,MSFT,-0.00764
1003,2017-12-29,MSFT,-0.00338


In [250]:
# to do so for all the returns dataframes, loop through delta_dict

#melt all the delta dataframes and store in list
melted_dfs = []
for key, delta_df in delta_dict.items():
    melted_dfs.append( delta_df.reset_index().melt(id_vars=['Date'], value_name=key))

In [251]:
# calculate 7-day returns after the date
return_df = pivoted_df.shift(-7) / pivoted_df - 1.0

# melt the return dataset and append to list
melted_dfs.append( return_df.reset_index().melt(id_vars=['Date'], value_name='return_7') )

In [252]:
# we now have 5 melted dataframes stred in the melted_dfs list
# now to join melted dataframes into a single ABT 

#merge two dataframes
pd.merge(melted_dfs[0], melted_dfs[1], on=['Date']).tail()

,Date,Name_x,delta_7,Name_y,delta_14
4011,2017-12-29,GOOGL,-0.02443,MSFT,0.01640
4012,2017-12-29,MSFT,-0.00338,AAPL,-0.00083
4013,2017-12-29,MSFT,-0.00338,AMZN,0.00643
4014,2017-12-29,MSFT,-0.00338,GOOGL,0.00383
4015,2017-12-29,MSFT,-0.00338,MSFT,0.01640


In [253]:
from functools import reduce

In [254]:
# create feature_dfs list that contains base features from original dataset plus melted dataset

# grab features from original dataset
base_df = df[['Date','Name', 'Volume', 'Close']]

# create a list with all the features dataframes
feature_dfs = [base_df] + melted_dfs

In [255]:
# now we can use reduce
# reduce applies to two arguments cummulatively objects and sequence 

# reduce-merge features into analytical bae table
abt = reduce(lambda left,right: pd.merge(left,right,on=['Date','Name']), feature_dfs)

# display examples from the ABT 
abt.tail(10)

,Date,Name,Volume,Close,delta_7,delta_14,delta_21,delta_28,return_7
994,2017-12-15,AMZN,"4,778,621.00000","1,179.14000",0.02325,-0.01396,0.04655,0.05218,0.00265
995,2017-12-18,AMZN,"2,947,625.00000","1,190.58000",0.02655,-0.00253,0.04686,0.06002,-0.00376
996,2017-12-19,AMZN,"2,587,792.00000","1,187.38000",0.02184,0.02248,0.05089,0.04811,-0.01508
997,2017-12-20,AMZN,"2,371,166.00000","1,177.62000",0.00744,0.00074,0.04556,0.04294,nan
998,2017-12-21,AMZN,"2,123,117.00000","1,174.76000",0.00831,0.01068,0.03095,0.04391,nan
999,2017-12-22,AMZN,"1,585,054.00000","1,168.36000",0.00363,0.03035,0.01055,0.03471,nan
1000,2017-12-26,AMZN,"2,005,187.00000","1,176.76000",0.00213,0.03083,-0.00779,0.03511,nan
1001,2017-12-27,AMZN,"1,867,208.00000","1,182.26000",0.00265,0.02596,-0.01135,0.04932,nan
1002,2017-12-28,AMZN,"1,841,676.00000","1,186.10000",-0.00376,0.02269,-0.00628,0.04292,nan
1003,2017-12-29,AMZN,"2,688,391.00000","1,169.47000",-0.01508,0.00643,0.00706,0.03504,nan
